In [1]:
import pandas as pd

In [2]:
match_df = pd.read_csv('./data/IPL_Matches_2008_2022.csv')
match_df.head()

,ID,City,Date,Season,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,SuperOver,WinningTeam,WonBy,Margin,method,Player_of_Match,Team1Players,Team2Players,Umpire1,Umpire2
0,1312200,Ahmedabad,2022-05-29,2022,Final,Rajasthan Royals,Gujarat Titans,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,bat,N,Gujarat Titans,Wickets,7.0,NaN,HH Pandya,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",CB Gaffaney,Nitin Menon
1,1312199,Ahmedabad,2022-05-27,2022,Qualifier 2,Royal Challengers Bangalore,Rajasthan Royals,"Narendra Modi Stadium, Ahmedabad",Rajasthan Royals,field,N,Rajasthan Royals,Wickets,7.0,NaN,JC Buttler,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...",CB Gaffaney,Nitin Menon
2,1312198,Kolkata,2022-05-25,2022,Eliminator,Royal Challengers Bangalore,Lucknow Super Giants,"Eden Gardens, Kolkata",Lucknow Super Giants,field,N,Royal Challengers Bangalore,Runs,14.0,NaN,RM Patidar,"['V Kohli', 'F du Plessis', 'RM Patidar', 'GJ ...","['Q de Kock', 'KL Rahul', 'M Vohra', 'DJ Hooda...",J Madanagopal,MA Gough
3,1312197,Kolkata,2022-05-24,2022,Qualifier 1,Rajasthan Royals,Gujarat Titans,"Eden Gardens, Kolkata",Gujarat Titans,field,N,Gujarat Titans,Wickets,7.0,NaN,DA Miller,"['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D ...","['WP Saha', 'Shubman Gill', 'MS Wade', 'HH Pan...",BNJ Oxenford,VK Sharma
4,1304116,Mumbai,2022-05-22,2022,70,Sunrisers Hyderabad,Punjab Kings,"Wankhede Stadium, Mumbai",Sunrisers Hyderabad,bat,N,Punjab Kings,Wickets,5.0,NaN,Harpreet Brar,"['PK Garg', 'Abhishek Sharma', 'RA Tripathi', ...","['JM Bairstow', 'S Dhawan', 'M Shahrukh Khan',...",AK Chaudhary,NA Patwardhan


In [3]:
ball_by_ball_df = pd.read_csv('./data/IPL_Ball_by_Ball_2008_2022.csv')
ball_by_ball_df.head()

,ID,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam
0,1312200,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
1,1312200,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals
2,1312200,1,0,3,JC Buttler,Mohammed Shami,YBK Jaiswal,NaN,1,0,1,0,0,NaN,NaN,NaN,Rajasthan Royals
3,1312200,1,0,4,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals
4,1312200,1,0,5,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals


In [4]:
motm = match_df['Player_of_Match'].value_counts()
motm

Player_of_Match
AB de Villiers    25
CH Gayle          22
DA Warner         18
RG Sharma         18
MS Dhoni          17
                  ..
CR Brathwaite      1
Mohsin Khan        1
A Zampa            1
BCJ Cutting        1
MF Maharoof        1
Name: count, Length: 262, dtype: int64

In [23]:
# Filter relevant columns for bowler analysis
bowler_data = ball_by_ball_df[['ID', 'bowler', 'total_run', 'isWicketDelivery', 'overs', 'kind']]

# Aggregate data by bowler
bowler_agg = bowler_data.groupby('bowler').agg(
    matches=('ID', 'nunique'),
    total_runs_conceded=('total_run', 'sum'),
    total_balls_bowled=('total_run', 'count'),
).reset_index()

# find total wickets which are not run out
wicket_data = bowler_data[(bowler_data['isWicketDelivery'] == 1) & (bowler_data['kind'] != 'run out')]

# Aggregate data by bowler
wicket_agg = wicket_data.groupby('bowler').agg(
    total_wickets=('isWicketDelivery', 'sum'),
    bowled=('kind', lambda x: (x == 'bowled').sum()),
    lbw=('kind', lambda x: (x == 'lbw').sum()),
    caught=('kind', lambda x: (x == 'caught').sum()),
    stumped=('kind', lambda x: (x == 'stumped').sum()),
    others=('kind', lambda x: ((x != 'bowled') & (x != 'lbw') & (x != 'caught') & (x != 'stumped')).sum()),
).reset_index()

# Merge wicket and run data
bowler_agg = bowler_agg.merge(wicket_agg, on='bowler', how='left')

bowler_agg['total_wickets'] = bowler_agg['total_wickets'].fillna(0).astype(int)

In [18]:
bowler_overs = ball_by_ball_df.groupby('bowler').apply(lambda x: x[['ID', 'overs']].drop_duplicates().shape[0]).reset_index(name='total_overs')
bowler_overs

,bowler,total_overs
0,A Ashish Reddy,45
1,A Badoni,2
2,A Chandila,39
3,A Choudhary,17
4,A Dananjaya,4
...,...,...
467,YK Pathan,193
468,YS Chahal,480
469,Yash Dayal,32
470,Yuvraj Singh,146


In [19]:
# map bowler overs
bowler_agg = bowler_agg.merge(bowler_overs, on='bowler')

In [20]:
# Calculate additional metrics for bowlers
bowler_agg['economy'] = bowler_agg['total_runs_conceded'] / (bowler_agg['total_overs'])
# bowler_agg['average'] = bowler_agg['total_runs_conceded'] / bowler_agg['total_wickets'].replace(0, 1)  # avoid division by zero

In [21]:
bowler_agg[bowler_agg['total_overs'] >= 20].sort_values(by='economy', ascending=True).head(10)

,bowler,innings,total_runs_conceded,total_balls_bowled,total_wickets,bowled,lbw,caught,stumped,others,total_overs,economy
277,Mohsin Khan,9,202,206,14,1.0,0.0,13.0,0.0,0.0,33,6.121212
2,A Chandila,12,245,234,11,0.0,0.0,4.0,2.0,5.0,39,6.282051
355,Rashid Khan,92,2411,2232,112,28.0,28.0,50.0,5.0,1.0,369,6.533875
425,Sohail Tanvir,11,275,265,22,8.0,3.0,11.0,0.0,0.0,42,6.547619
159,IS Sodhi,8,204,182,9,1.0,1.0,7.0,0.0,0.0,31,6.580645
398,SMSM Senanayake,8,211,195,9,4.0,3.0,1.0,1.0,0.0,32,6.593750
131,FH Edwards,6,160,150,5,1.0,1.0,3.0,0.0,0.0,24,6.666667
397,SM Pollock,13,307,280,11,0.0,1.0,9.0,1.0,0.0,46,6.673913
6,A Kumble,42,1089,983,45,8.0,4.0,24.0,8.0,1.0,163,6.680982
263,MM Ali,37,631,564,24,6.0,4.0,13.0,0.0,1.0,94,6.712766


In [22]:
bowler_agg[bowler_agg['bowler'].str.contains('Bumrah')]

,bowler,innings,total_runs_conceded,total_balls_bowled,total_wickets,bowled,lbw,caught,stumped,others,total_overs,economy
177,JJ Bumrah,120,3499,2857,148,33.0,16.0,95.0,0.0,4.0,463,7.557235
